In [1]:
##---------------------------------------------------------------------------------
## Summary : perform parsing using tree parser
## Author  : Srinivas Venkata Vemparala
## Source  : https://github.com/neubig/nn4nlp-code
##---------------------------------------------------------------------------------

# paper : http://aclweb.org/anthology/P/P15/P15-1150.pdf

import codecs
import time
import random
import dynet as dy
import numpy as np
import re

from collections import defaultdict, Counter


In [2]:
# methods to extract the tree from the file

# tokenize the line
def _tokenize_sexpr(s):
    tokker = re.compile(r" +|[()]|[^ ()]+")
    toks = [t for t in [match.group(0) for match in tokker.finditer(s)] if t[0] != " "]
    
    return toks

# get the stuff in the brackets
def _within_bracket(toks):
    label = next(toks)
    children = []

    for tok in toks:
        if tok == "(":
            children.append(_within_bracket(toks))
        elif tok == ")":
            return Tree(label, children)
        else: 
            children.append(Tree(tok, None))
            
    assert(False),list(toks)

In [3]:
# the tree class
class Tree(object):
    
    def __init__(self, label, children=None):
        self.label = label
        self.children = children
        
    @staticmethod
    def from_sexpr(string):
        toks = iter(_tokenize_sexpr(string))
        assert next(toks) == "("
        return _within_bracket(toks)
    
    def __str__(self):
        if self.children is None: return self.label
        return "[%s %s]" % (self.label, " ".join([str(c) for c in self.children]))

    def isleaf(self): 
        return self.children==None
    
    def leaves_iter(self):
        if self.isleaf():
            yield self
        else:
            for c in self.children:
                for l in c.leaves_iter(): 
                    yield l

    def leaves(self): 
        return list(self.leaves_iter())

    
    def nonterms_iter(self):
        if not self.isleaf():
            yield self
            for c in self.children:
                for n in c.nonterms_iter():
                    yield n
          
    def nonterms(self): 
        return list(self.nonterms_iter())

In [4]:
# method to read the dataset
def read_dataset(filename):
    return [Tree.from_sexpr(line.strip()) for line in codecs.open(filename,"r")]


In [5]:
# method to get the vocabulary and convert it to int and back to words
def get_vocabs(trees):
    label_vocab = Counter()
    word_vocab  = Counter()
    
    for tree in trees:
        label_vocab.update([n.label for n in tree.nonterms()])
        word_vocab.update([l.label for l in tree.leaves()])
        
    labels = [x for x,c in label_vocab.items() if c > 0]
    words  = ["_UNK_"] + [x for x,c in word_vocab.items() if c > 0]
    
    l2i = {l:i for i,l in enumerate(labels)}
    w2i = {w:i for i,w in enumerate(words)}

    return l2i, w2i, labels, words

In [6]:
# read the training and dev data
train = read_dataset("../data/parsing/trees/train.txt")
dev = read_dataset("../data/parsing/trees/dev.txt")

In [7]:
l2i, w2i, i2l, i2w = get_vocabs(train)

ntags = len(l2i)
nwords = len(w2i)

print('number of words : ',nwords)
print('number of tags : ',ntags)

number of words :  18281
number of tags :  5


In [8]:
# lets define a the Tree RNN described in the paper
# normal RNN
class TreeRNNBuilder(object):
    
    def __init__(self, model, word_vocab, hdim):
        self.W = model.add_parameters((hdim, 2*hdim))
        self.E = model.add_lookup_parameters((len(word_vocab),hdim))
        self.w2i = word_vocab
    
    def expr_for_tree(self, tree):
        # if we are looking at leaf   
        if tree.isleaf():
            return self.E[self.w2i.get(tree.label,0)]

        # if the tree has only one child then that child must be leaf
        if len(tree.children) == 1:
            assert(tree.children[0].isleaf())
            expr = self.expr_for_tree(tree.children[0])

            return expr
        
        # if the tree has two children
        assert(len(tree.children) == 2),tree.children[0]
        e1 = self.expr_for_tree(tree.children[0])
        e2 = self.expr_for_tree(tree.children[1])
        W = dy.parameter(self.W)
        expr = dy.tanh(W*dy.concatenate([e1,e2]))

        return expr
    

In [9]:
# RNN with the LSTM structure
class TreeLSTMBuilder(object):

    def __init__(self, model, word_vocab, wdim, hdim):
        self.WS = [model.add_parameters((hdim, wdim)) for _ in "iou"]    # input output update
        self.US = [model.add_parameters((hdim, 2*hdim)) for _ in "iou"]  # input output update
        self.UFS =[model.add_parameters((hdim, hdim)) for _ in "ff"]     # forget gate
        self.BS = [model.add_parameters(hdim) for _ in "iouf"]           # input output update and forget
        self.E = model.add_lookup_parameters((len(word_vocab),wdim))
        self.w2i = word_vocab



    def expr_for_tree(self, tree):
        # if we are looking at leaf  
        if tree.isleaf():
            return self.E[self.w2i.get(tree.label,0)]
        
        # if the tree has only one child then that child must be leaf
        if len(tree.children) == 1:
            assert(tree.children[0].isleaf())
            emb = self.expr_for_tree(tree.children[0])
            Wi,Wo,Wu   = [dy.parameter(w) for w in self.WS]
            bi,bo,bu,_ = [dy.parameter(b) for b in self.BS]

            i = dy.logistic(Wi*emb + bi)
            o = dy.logistic(Wo*emb + bo)
            u = dy.tanh(    Wu*emb + bu)
            c = dy.cmult(i,u)

            expr = dy.cmult(o,dy.tanh(c))
            return expr

        # if the tree has two children
        assert(len(tree.children) == 2),tree.children[0]
        e1 = self.expr_for_tree(tree.children[0])
        e2 = self.expr_for_tree(tree.children[1])
        Ui,Uo,Uu = [dy.parameter(u) for u in self.US]
        Uf1,Uf2 = [dy.parameter(u) for u in self.UFS]
        bi,bo,bu,bf = [dy.parameter(b) for b in self.BS]
        e = dy.concatenate([e1,e2])
        i = dy.logistic(Ui*e + bi)
        o = dy.logistic(Uo*e + bo)
        f1 = dy.logistic(Uf1*e1 + bf)
        f2 = dy.logistic(Uf2*e2 + bf)
        u = dy.tanh(    Uu*e + bu)
        c = dy.cmult(i,u) + dy.cmult(f1,e1) + dy.cmult(f2,e2)
        h = dy.cmult(o,dy.tanh(c))
        expr = h

        return expr

In [10]:
# lets define the model and the optimizer
model = dy.Model()
trainer = dy.AdamTrainer(model)

# lets define the Hiddenlayer size and embedding size
EMB_SIZE = 64
HID_SIZE = 64

In [17]:
# lets first execute the childsum LSTM
builder = TreeRNNBuilder(model, w2i, HID_SIZE)

W_sm = model.add_parameters((ntags, HID_SIZE))        # Softmax weights
b_sm = model.add_parameters((ntags))                  # Softmax bias


# lets write a method to compute the scores 
def computeScores(tree):
    # renew the computation graph
    dy.renew_cg()
    
    # get the embedding
    emb = builder.expr_for_tree(tree)
    
    # get the softmax weights into the cg
    weights = dy.parameter(W_sm)
    biases = dy.parameter(b_sm)
    
    return weights*emb+biases

In [18]:
# lets perform Training
print('Started Training .....')

for i in range(100):
    # randomly shuffle the training examples
    random.shuffle(train)
    
    trainLoss = 0
    startTime = time.time()
    for tree in train:
        myLoss = dy.hinge(computeScores(tree), l2i[tree.label]) # can pick neg logsoftmaxloss also
        
    trainLoss += myLoss.value()
    myLoss.backward()
    trainer.update()
    print("Train_loss at iter : ",i," is ",trainLoss/len(train),'. Time taken : ',(-startTime+time.time()))
    
    startTime = time.time()
    correct = 0
    for tree in dev:
        scores = computeScores(tree).npvalue()
        predict = np.argmax(scores)
        if(predict==l2i[tree.label]):
            correct += 1
    print("Test_Accuracy at iter : ",i," is ",correct/len(dev),'. Time taken : ',(-startTime+time.time()))

Started Training .....
Train_loss at iter :  0  is  2.0867597297782756e-05 . Time taken :  1.0637664794921875
Test_Accuracy at iter :  0  is  0.26612170753860126 . Time taken :  0.560774564743042
Train_loss at iter :  1  is  0.0 . Time taken :  0.8954987525939941
Test_Accuracy at iter :  1  is  0.2652134423251589 . Time taken :  0.5678150653839111
Train_loss at iter :  2  is  0.00044717195998416855 . Time taken :  0.8768372535705566
Test_Accuracy at iter :  2  is  0.2652134423251589 . Time taken :  0.5650622844696045
Train_loss at iter :  3  is  0.0004942372497101402 . Time taken :  0.874129056930542
Test_Accuracy at iter :  3  is  0.2670299727520436 . Time taken :  0.5331592559814453
Train_loss at iter :  4  is  0.00029265615489152487 . Time taken :  0.8747680187225342
Test_Accuracy at iter :  4  is  0.2670299727520436 . Time taken :  0.5432217121124268
Train_loss at iter :  5  is  0.0006952050808217195 . Time taken :  0.8548574447631836
Test_Accuracy at iter :  5  is  0.2652134423251

Test_Accuracy at iter :  47  is  0.26067211625794734 . Time taken :  0.5312485694885254
Train_loss at iter :  48  is  3.3446179514520624e-05 . Time taken :  0.8750247955322266
Test_Accuracy at iter :  48  is  0.259763851044505 . Time taken :  0.562518835067749
Train_loss at iter :  49  is  0.00018073844440867393 . Time taken :  1.0788524150848389
Test_Accuracy at iter :  49  is  0.2579473206176203 . Time taken :  0.5468764305114746
Train_loss at iter :  50  is  0.0005818571379122217 . Time taken :  0.875009298324585
Test_Accuracy at iter :  50  is  0.25522252497729336 . Time taken :  0.919074535369873
Train_loss at iter :  51  is  0.00014023071203785444 . Time taken :  0.9343366622924805
Test_Accuracy at iter :  51  is  0.2579473206176203 . Time taken :  0.6249997615814209
Train_loss at iter :  52  is  0.00033862997641724145 . Time taken :  1.4417426586151123
Test_Accuracy at iter :  52  is  0.25885558583106266 . Time taken :  0.8760025501251221
Train_loss at iter :  53  is  0.00059476

Train_loss at iter :  95  is  2.3444514540250828e-05 . Time taken :  1.0094079971313477
Test_Accuracy at iter :  95  is  0.2670299727520436 . Time taken :  0.5968325138092041
Train_loss at iter :  96  is  1.4973043948970037e-05 . Time taken :  1.0244693756103516
Test_Accuracy at iter :  96  is  0.2670299727520436 . Time taken :  0.6037271022796631
Train_loss at iter :  97  is  0.00035746270016338047 . Time taken :  1.010801076889038
Test_Accuracy at iter :  97  is  0.26430517711171664 . Time taken :  0.6096444129943848
Train_loss at iter :  98  is  0.0006299770615074072 . Time taken :  1.0105443000793457
Test_Accuracy at iter :  98  is  0.26430517711171664 . Time taken :  0.5954718589782715
Train_loss at iter :  99  is  9.908986169747199e-05 . Time taken :  1.1234097480773926
Test_Accuracy at iter :  99  is  0.26612170753860126 . Time taken :  0.6000771522521973


In [16]:
builder = TreeLSTMBuilder(model, w2i, HID_SIZE, EMB_SIZE)

# lets perform Training
print('Started Training .....')

for i in range(100):
    # randomly shuffle the training examples
    random.shuffle(train)
    
    trainLoss = 0
    startTime = time.time()
    for tree in train:
        myLoss = dy.hinge(computeScores(tree), l2i[tree.label]) # can pick neg logsoftmaxloss also
        
    trainLoss += myLoss.value()
    myLoss.backward()
    trainer.update()
    print("Train_loss at iter : ",i," is ",trainLoss/len(train),'. Time taken : ',(-startTime+time.time()))
    
    startTime = time.time()
    correct = 0
    for tree in dev:
        scores = computeScores(tree).npvalue()
        predict = np.argmax(scores)
        if(predict==l2i[tree.label]):
            correct += 1
    print("Test_Accuracy at iter : ",i," is ",correct/len(dev),'. Time taken : ',(-startTime+time.time()))

Started Training .....
Train_loss at iter :  0  is  0.0005263689528690295 . Time taken :  10.181970834732056
Test_Accuracy at iter :  0  is  0.26248864668483196 . Time taken :  3.2994637489318848
Train_loss at iter :  1  is  0.0008902569835105639 . Time taken :  10.508680820465088
Test_Accuracy at iter :  1  is  0.26248864668483196 . Time taken :  3.0892579555511475
Train_loss at iter :  2  is  0.0008759815594676729 . Time taken :  10.083086252212524
Test_Accuracy at iter :  2  is  0.26248864668483196 . Time taken :  3.188868761062622
Train_loss at iter :  3  is  0.0005096780896633305 . Time taken :  10.141981363296509
Test_Accuracy at iter :  3  is  0.26248864668483196 . Time taken :  3.099905252456665
Train_loss at iter :  4  is  0.00019126700774560706 . Time taken :  10.164716720581055
Test_Accuracy at iter :  4  is  0.26248864668483196 . Time taken :  3.0974981784820557
Train_loss at iter :  5  is  0.00019399816177311015 . Time taken :  10.246028184890747
Test_Accuracy at iter :  5

Train_loss at iter :  94  is  0.0005643919166107749 . Time taken :  4.851523160934448
Test_Accuracy at iter :  94  is  0.26067211625794734 . Time taken :  1.4219129085540771
Train_loss at iter :  95  is  0.0001289344738038738 . Time taken :  4.968792200088501
Test_Accuracy at iter :  95  is  0.2633969118982743 . Time taken :  1.4530854225158691
Train_loss at iter :  96  is  0.0006505682897031977 . Time taken :  4.734372615814209
Test_Accuracy at iter :  96  is  0.2652134423251589 . Time taken :  1.3437483310699463
Train_loss at iter :  97  is  0.00018254042453087223 . Time taken :  4.734376907348633
Test_Accuracy at iter :  97  is  0.2615803814713896 . Time taken :  1.3906264305114746
Train_loss at iter :  98  is  0.0007279327188091778 . Time taken :  4.515620470046997
Test_Accuracy at iter :  98  is  0.2724795640326976 . Time taken :  1.3906290531158447
Train_loss at iter :  99  is  0.00014138481255327717 . Time taken :  4.578124761581421
Test_Accuracy at iter :  99  is  0.27247956403